In [2]:
import pandas as pd
import nltk
import re  # 정규식을 위한 모듈
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import string
from nltk.corpus import stopwords
import pyLDAvis
from pyLDAvis import prepare  # pyLDAvis에서 prepare 직접 사용
import matplotlib.pyplot as plt

# nltk 데이터 다운로드 (불용어 목록 다운로드)
nltk.download('stopwords')

# 데이터 로드
file_path = r"C:\Users\hwjh2\Desktop\딜리버리엠\인도 DB_표준화_210909.xlsx"
data = pd.read_excel(file_path)

# Product 열만 사용
products = data['Product'].dropna().tolist()

# 데이터 전처리: 텍스트 소문자화, 구두점 제거, 불용어 제거, 숫자 및 용량 제거
def preprocess_text(text):
    # 소문자화
    text = text.lower()
    # 숫자와 용량 제거 (예: "100ml", "200g", "1kg") + 숫자만 제거
    text = re.sub(r'\b\d+\s?(ml|g|kg|l|mg|oz|cm|mm|floz)?\b', '', text)
    # 구두점 제거
    text = ''.join([char for char in text if char not in string.punctuation])
    # 불용어 제거
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# 모든 텍스트에 대해 전처리 수행
processed_products = [preprocess_text(product) for product in products]

# 텍스트를 벡터화 (Bag of Words)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(processed_products)

# K값을 시각화 결과에 따라 선택하고, 그 값을 사용하여 LDA 모델링
num_topics_optimal = 10  # 엘보우 기법에서 얻은 최적 K값
lda_optimal = LatentDirichletAllocation(n_components=num_topics_optimal, random_state=42)
lda_optimal.fit(X)

# pyLDAvis 시각화 준비
# 어휘(vocab) 추출
vocab = vectorizer.get_feature_names_out()

# 문서-토픽 분포 추출
doc_topic_dists = lda_optimal.transform(X)

# 토픽-단어 분포 추출
topic_term_dists = lda_optimal.components_

# 단어 빈도 계산
term_frequency = X.sum(axis=0).A1

# 시각화 준비
vis = pyLDAvis.prepare(
    topic_term_dists=topic_term_dists, 
    doc_topic_dists=doc_topic_dists, 
    doc_lengths=X.sum(axis=1).A1, 
    vocab=vocab, 
    term_frequency=term_frequency
)

# 시각화 HTML로 저장
pyLDAvis.save_html(vis, 'lda_visualization10.html')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hwjh2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
